In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm
import random
import gzip
random.seed(64)
import gc

In [2]:
list_cand_past = [
#     "popularity_clicks", # clicks回数上位
#     "popularity_carts", # carts回数上位
#     "popularity_orders", # orders回数上位
    "past_clicks", # 過去clicks実績あり
    "past_carts", # 過去carts実績あり
    "past_orders", # 過去orders実績あり
    # "association_clicks", # clicksのアソシエーションルール
    # "association_carts", # cartsのアソシエーションルール
    # "association_orders", # ordersのアソシエーションルール
#     "popularity_clicks_after",
    "popularity_carts_after",
#     "popularity_orders_after",

]
list_cand_rule = [
    "imf_clicks", # IMFの推薦aid
#     "imf_carts", # IMFの推薦aid
    # "imf_orders", # IMFの推薦aid
#     "bpr_clicks", # IMFの推薦aid
#     "bpr_carts", # IMFの推薦aid
    # "bpr_orders", # IMFの推薦aid
    "bpr_clicksxcarts", # IMFの推薦aid
#     "bpr_cartsxorders", # IMFの推薦aid
#     "bpr_ordersxclicks", # IMFの推薦aid
    # "prone_clicks",
    # "prone_carts",
    # "prone_orders",

]
list_cand_all = list_cand_past + list_cand_rule

In [3]:
path_input = {}
path_input_label = {}
path_input_past = {}
path_input_popul = {}
path_input_rank = {}
path_input_assoc = {}
path_input_cf = {}

path_output = {}
for i in ["train1","valid"]:
    path_input[i] = f"/kaggle/input/otto-make-data-train-test-split/{i}.jsonl"
    path_input_label[i] = f"/kaggle/input/otto-make-data-label/label_{i}.parquet"
    path_input_past[i] = f"/kaggle/input/otto-make-data-past-event/past_event_{i}.pickle"
    path_input_popul[i] = f"/kaggle/input/otto-make-data-past-event/population_{i}.pickle"
    path_input_assoc[i] = f"/kaggle/input/otto-make-data-past-event/association_{i}.pickle"
    path_input_rank[i] = f"/kaggle/input/otto-feat-count-by-time/rank_by_time_{i}.pickle"
    path_output[i] = f"candidate_{i}.jsonl.gz"
    path_input_cf[i] = {}
    for cand_rule in list_cand_rule:
        algo, types = cand_rule.split('_')
        path_input_cf[i][cand_rule] = f"/kaggle/input/otto-cf-{algo}-{types}/{algo}_rules_{types}_{i}.pickle"   
    
sample_size = 100000
debug = False

n_session = {"train1": 2237925, "valid": 1735763}
time_min = {"train1":1660514400152, "valid": 1661119200060}
time_max = {"train1": 1661119199547, "valid": 1661723998043}

In [4]:
# event実績のあるaid+アソシエーションルールに該当するaid
cnt_u20 = 0
for i in ["train1", "valid"]:
#     # 人気
#     popul = pd.read_pickle(path_input_popul[i])
    # 過去実績あり
    past = pd.read_pickle(path_input_past[i])
    # # アソシエーション
    # assoc = pd.read_pickle(path_input_assoc[i])
    cf = {}
    # session後の人気
    rank_after = pd.read_pickle(path_input_rank[i])
    for idx in range(len(rank_after)):
        for types in ["clicks", "carts", "orders"]:
            rank_after[idx][types] = [y[0] for y in sorted(rank_after[idx][types].items(), key = lambda x : x[1])[:20]]
    file = gzip.open(path_output[i], "wt")
    if i != "test":
        # カバー率計算用
        n_grand_truth = {"clicks": 0, "carts": 0, "orders": 0}
        n_hit = {x: {"clicks": 0, "carts": 0, "orders": 0} for x in list_cand_all + ["all"]}
        df_label = pd.read_parquet(path_input_label[i]) # 正解ラベル
        dict_label = {}
        for types in ["clicks", "carts", "orders", "any"]:
            dict_label[types] = dict(zip(df_label["session"], df_label[f"label_{types}"]))
    chunks = pd.read_json(path_input[i], lines = True, chunksize = sample_size)
    tot_aid = 0
    for n_chunk, df_chunk in enumerate(tqdm(chunks, total = n_session[i]//sample_size+1)):
        for cand_rule in list_cand_rule:
            cf[cand_rule] = pd.read_pickle(path_input_cf[i][cand_rule])
            cf[cand_rule] = {key: cf[cand_rule][key] for key in set(cf[cand_rule].keys()) & set(df_chunk["session"])}
            
        for j, session in df_chunk.iterrows():
            cand_all = set()
            session = session.to_dict()
            ses = session["session"]
            session["candidate"] = {}
            list_event = session["events"][:session["cutoff"]] # cutoff以前のeventのみ使える
            last_ts = list_event[-1]["ts"]
            idx_box = (last_ts - time_min[i]) // (1000*60*60*3)
            session["candidate"][f"popularity_carts_after"] = []

            # event実績のあるaid+アソシエーションルール
            for types in ["clicks", "carts", "orders"]:
                # event実績のあるaid
                session["candidate"][f"past_{types}"] = past[types].get(ses, [])
                cand_all = cand_all | set(session["candidate"][f"past_{types}"])
#                 # 人気
#                 session["candidate"][f"popularity_{types}"] = popul[types]
#                 session["candidate"][f"popularity_{types}_after"] = rank_after[idx_box][types]
                
            for cand_rule in list_cand_rule:
                session["candidate"][cand_rule] = cf[cand_rule].get(ses,[])
                cand_all = cand_all | set(session["candidate"][cand_rule])
            n_add = 0
            while len(cand_all) < 20:
                aid = rank_after[idx_box]["carts"][n_add]
                session["candidate"][f"popularity_carts_after"] += [aid]
                cand_all.add(aid)
                n_add += 1
                
            file.write(json.dumps(session) + "\n")
            if i != "test":
                # カバー率を算出
                cand_all = set()
                for key in list_cand_all:
                    cand_all.update(session["candidate"][key])
                tot_aid += len(cand_all)
                if len(cand_all) < 20: cnt_u20 += 1
                for types in ["clicks", "carts", "orders"]:
                    aid_truth = dict_label[types][session["session"]]
                    n_grand_truth[types] += len(aid_truth)
                    n_hit["all"][types] += sum([1 for x in aid_truth if x in cand_all]) 
                    for rule in list_cand_all:
                        set_cand_rule = set(session["candidate"][rule])
                        n_hit[rule][types] += sum([1 for x in aid_truth if x in set_cand_rule]) 
            if debug and j == 10: break
        if debug: break
        gc.collect()
    file.close()
    if i != "test":
        # カバー率
        for types in ["clicks", "carts", "orders"]:
            for rule in list_cand_all + ["all"]:
                rate = n_hit[rule][types]/n_grand_truth[types] if n_grand_truth[types] != 0 else 0
                print(f"{types}, {rule}: {rate*100:.3f}")
        if debug: break
        print(f"num_session: {n_session[i]}, num_candidate: {tot_aid}, avg_candidate: {tot_aid/n_session[i]}, u20: {cnt_u20}")

  0%|          | 0/23 [00:00<?, ?it/s]

clicks, past_clicks: 20.599
clicks, past_carts: 3.307
clicks, past_orders: 0.355
clicks, popularity_carts_after: 0.003
clicks, imf_clicks: 12.696
clicks, bpr_clicksxcarts: 11.439
clicks, all: 37.438
carts, past_clicks: 29.579
carts, past_carts: 6.295
carts, past_orders: 0.543
carts, popularity_carts_after: 0.004
carts, imf_clicks: 7.267
carts, bpr_clicksxcarts: 7.217
carts, all: 40.282
orders, past_clicks: 51.311
orders, past_carts: 42.767
orders, past_orders: 1.421
orders, popularity_carts_after: 0.001
orders, imf_clicks: 4.928
orders, bpr_clicksxcarts: 4.051
orders, all: 64.426
num_session: 2237925, num_candidate: 80526694, avg_candidate: 35.98274919847627, u20: 0


  0%|          | 0/18 [00:00<?, ?it/s]

clicks, past_clicks: 31.783
clicks, past_carts: 4.309
clicks, past_orders: 0.604
clicks, popularity_carts_after: 0.005
clicks, imf_clicks: 16.051
clicks, bpr_clicksxcarts: 13.928
clicks, all: 52.273
carts, past_clicks: 30.255
carts, past_carts: 6.859
carts, past_orders: 0.573
carts, popularity_carts_after: 0.004
carts, imf_clicks: 7.332
carts, bpr_clicksxcarts: 7.066
carts, all: 40.981
orders, past_clicks: 51.891
orders, past_carts: 43.406
orders, past_orders: 1.551
orders, popularity_carts_after: 0.003
orders, imf_clicks: 4.789
orders, bpr_clicksxcarts: 3.932
orders, all: 65.133
num_session: 1735763, num_candidate: 64290854, avg_candidate: 37.03895865967877, u20: 0


In [5]:
def process_session():
            ses = session["session"]
            session["candidate"] = {}
            # event実績のあるaid
            session["candidate"]["past_event"] = past.get(ses, [])
            # event実績のあるaid+アソシエーションルール
            for types in ["clicks", "carts", "orders"]:
                # 人気
                session["candidate"][f"popularity_{types}"] = popul[types]
                # アソシエーションルール
                session["candidate"][f"association_{types}"] = assoc[types].get(ses, [])
                # IMF
                session["candidate"][f"imf_{types}"] = imf[types].get(ses,[])
            if i == "train":
                # 負例のみサンプリング
                for key in session["candidate"].keys():
                    list_cand = session["candidate"][key]
                    set_truth = set(dict_label["any"][ses])
                    session["candidate"][key] = [x for x in list_cand if x in set_truth or random.random() < sampling_rate]
#             file.write(json.dumps(session) + "\n")
            # カバー率を算出
            cand_all = set()
            for key in session["candidate"].keys():
                cand_all.update(session["candidate"][key])
            for types in ["clicks", "carts", "orders"]:
                aid_truth = dict_label[types][ses]
                n_grand_truth[types] += len(aid_truth)
                n_hit["all"][types] += sum([1 for x in aid_truth if x in cand_all]) 
                for rule in list_cand_rule:
                    set_cand_rule = set(session["candidate"][rule])
                    n_hit[rule][types] += sum([1 for x in aid_truth if x in set_cand_rule]) 


In [6]:
if debug:
    import line_profiler
    pr = line_profiler.LineProfiler()#LineProfilerクラスのインスタンス生成
    pr.add_function(process_session)#対象の関数をadd_functionで登録
    pr.runcall(process_session)#runcallで実行
    pr.print_stats()#結果を表示